### **FITNETS: Hints For Thin Deep Nets**

#### **摘要**

- 更深的网络性能更高，但是训练更困难，因为**更深的网络往往更加非线性**
- 本文基于 **Knowledge Distillation** 的思想，提出了一种**相对于教师网络训练更深但更窄**的网络的方法
- 不仅使用**输出**，还使用教师的**中间层输出**作为监督信号
- 这样训练出来的学术网络，泛化性更好，收敛更快，**这是一种由所选学生容量所控制的权衡**

-----------------------------------
#### **引言**
- 分类的 top-performing 的网络都又宽又深，具有大量参数。**在推理阶段，这些网络由于大量的乘法运算而非常耗时**，并且很占用内存
- 如果**对内存和时间进行限制**，宽而深的网络就稍显乏力了

-------------------------------------

> 模型压缩的前人贡献：

- In Bucila et al. (2006), 作者建议**训练神经网络，来模拟大模型或者集成模型的结果**。具体而言，通过大模型或者集成模型标记无标记的数据，之后让神经网络去学习，从而达到类似精度
- In Ba & Caruana (2014)，**将网络压缩的更浅但更宽**
- Knowledge Distillation (KD) (Hinton & Dean, 2014)，~~二者深度类似~~
- 一言以蔽之，**前人未对深度做文章**

-------------------------------------
- **前人已证：网络深一些比较好**
- **前人已证：越深越难训练，因为它们由连续的非线性组成，从而导致高度非凸和非线性函数**
- 在优化问题上，已有前辈做出种种努力：原文有列举
- 本文 **care 深度问题**，提出了一种**相对于教师网络训练更深但更窄**的网络的方法，网络名称为 **FitNets**
- **一言以蔽之，KD 学习结果，本文学习中间表示特征**
- 深有助于泛化，窄有助于减少计算
- **MNIST, CIFAR-10, CIFAR-100, SVHN and AFLW benchmark datasets**

---------------------------------------
#### **方法**
- ~~KD 中 student 和 teacher 深度近似~~，作者说**如果让 student 更深一些会更好（than their teacher），问题是不好做优化**
- 为此，作者引入了 teacher network 的 **hints**。
  - *A hint is defined as the output of a teacher's hidden layer responsible for guiding the student's learning process.*
- 这被称为一对对 **hint/guided layer。一对 hint/guided layer** 的目的是，**让 student 的 hint 和 teacher 的 guided layer 的输出尽可能接近**
- **拥有 hint 对于 student network 而言可以理解为一种形式的正则，因此，我们要保证对于 student network 不要过度正则**
- 设置引导层越深，我们给网络提供的灵活性就越低，因此，FitNets 更有可能遭受过度正则化
- **作者选择了二者的 middle layer**

--------------------------------------------
- Given that the teacher network will usually be wider than the FitNet, the selected hint layer may have **more outputs** than the guided layer. **For that reason, we add a regressor** to the guided layer, whose output matches the size of the hint layer. **Then, we train the FitNet parameters from the first layer up to the guided layer as well as the regressor parameters** by minimizing the following loss function:

$$\mathcal{L}_{HT}(\mathbf{W}_{Guided}, \mathbf{W}_r) = \frac{1}{2}||u_h(\mathbf{x}; \mathbf{W_{Hint}}) - r(v_g(\mathbf{x}; \mathbf{W_{Guided}}); \mathbf{W_r})||^2$$

- where $u_h$ and $v_g$ are the teacher/student deep nested functions up to their respective hint/guided layers with parameters $\mathbf{W_{Hint}}$ and $\mathbf{W_{Guided}}$, $r$ is the regressor function on top of the guided layer with parameters $\mathbf{W_r}$. Note that the outputs of $u_h$ and $r$ have to be comparable, i.e., $u_h$ and $r$ must be the same non-linearity.


---------------------------------------------

> 但是，如果 *hint/guided layer* 是卷积层，那么使用全连接层的 *regressor* 会导致参数过多，为解决这一问题，作者使用 *convolutional regressor* 来进行替代，这会导致更少的参数。**具体见原文**


------------------------------------------------------------

#### **按阶段训练**

![Training a student network using hint](../img/Training-a-student-network-using-hints.png)

1. 首先，要有一个已经训练好的 teacher network，对 FitNet 进行随机初始化。**对应于上图（a）**
2. 在 FitNet 的 guided layer 之上添加一个由参数 $\mathbf{W_r}$ 参数化的回归器
3. 训练 guided layer 和 regressor，使得 hint 和 guided layer 的输出尽可能接近。**对应于上图（b）**，最小化如下损失

$$\mathcal{L}_{HT}(\mathbf{W}_{Guided}, \mathbf{W}_r) = \frac{1}{2}||u_h(\mathbf{x}; \mathbf{W_{Hint}}) - r(v_g(\mathbf{x}; \mathbf{W_{Guided}}); \mathbf{W_r})||^2$$

4. **基于预训练的参数，训练整个** FitNet 的参数 $\mathbf{W_s}$，使得 FitNet 的输出和 teacher 的输出尽可能接近。**对应于上图（c）**，最小化 KD 中所提出的损失

具体算法如下所示：

![Algorithm 1: FitNet Stage-Wise Training](../img/Algorithm-1：FitNet-Stage-Wise-Training.png)


- 作者认为，**FitNet 的训练过程可以看作是 Curriculum Learning** 的一种特别形式，**Curriculum learning extension (Gulcehre & Bengio, 2013)表明，在训练期间使用中间层的指导提示，可以大大简化训练**

- **在第二阶段，也就是训练整个 FitNet 的阶段，我们对于 $\lambda$ 进行线性的退火**

----------------------------------
#### **基准数据集的结果**
- [code](https://github.com/adri-romsor/FitNets)

##### **CIFAR-10 and CIFAR-100**
- 参照 Goodfellow et al. (2013b), 作者进行了如下预处理：
  - **对比归一化（contrast normalization）**：将数据的均值和方差归一化到 [0, 1] 之间。这有助于消除数据的不同尺度对模型的影响。
  - **ZCA 白化（ZCA whitening）**：将数据的协方差矩阵化为单位矩阵。这有助于消除数据之间的相关性，使模型更容易学习到数据的潜在结构。
- 参照 Goodfellow et al. (2013b); Chen-Yu et al. (2014), 作者在训练的过程中进行了 **随机翻转来增强数据**

- 实验细节见原文，结果如下：

![Accuracy on CIFAR](../img/Table-1：Accuracy-on-CIFAR.png)

> **效果优于 teacher model，甚至比着最优的方法，结果也基本不弱于人，但是模型参数量更小**，表明越深越好

> **作者提出了另外四种架构，见原文，但是效果不如 FitNet**

-------------------------------
##### **SVHN and MNIST**

![](../img/SVHN-and-MNIST-error.png)

---------------------------------
##### **AFLW**

AFLW 是一个真实世界的**人脸数据库**，包含 25,000 张带注释的图像。为了在人脸识别设置中评估该框架，作者从图像中提取了 **16x16** 像素的正样本，**即包含人脸的区域**。作者还从 ImageNet 数据集（Russakovsky 等人，2014 年）中提取了 25,000 个**不包含人脸的 16x16 像素的负样本**。作者使用了 90% 的提取的图像来训练网络。

以下是一些关键点：
- AFLW 是一个真实世界的人脸数据库，包含 25,000 张带注释的图像。
- 作者从图像中提取了 16x16 像素的正样本，即包含人脸的区域。
- 作者还从 ImageNet 数据集（Russakovsky 等人，2014 年）中提取了 25,000 个不包含人脸的 16x16 像素的负样本。
- 作者使用了 90% 的提取的图像来训练网络


--------------------------------------
#### **实验结果分析**
- HT 和 KD 之间的**唯一区别**是参数空间的起点
- **HT 是比 KD 更强的正则化器，因为它在测试集上带来更好的泛化性能**
- 网络越深表现越好


FitNets 的效率和性能的权衡

- FitNets 是通过在网络中添加额外的“hints”层来压缩网络的。作者在 CIFAR-10 数据集上对 FitNets 进行了实验，并将其与教师模型进行了比较。

- 实验结果表明，FitNets 可以显著提高网络的效率，同时也能保持较高的性能。例如，FitNet 1 比教师模型快了一个数量级，但只损失了 1.3% 的性能。FitNet 2 比教师模型快了 4.64 倍，并取得了 0.9% 的性能提升。

- 作者还提到，FitNets 可以与其他方法相结合，进一步提高效率和性能。例如，可以使用矩阵分解来加速 FitNets 的卷积层，或者使用量化技术来减少存储要求。

- 总体而言，FitNets 是一种有效的网络压缩方法，可以显著提高网络的效率和性能。

- 以下是一些关键点：
    - FitNets 通过在网络中添加额外的“hints”层来压缩网络。
    - FitNets 可以显著提高网络的效率，同时也能保持较高的性能。
    - FitNets 可以与其他方法相结合，进一步提高效率和性能。

--------------------------------
#### **结论**
- **将窄而深的网络的内部层与教师网络的隐藏状态进行提示，比将它们与分类目标进行提示具有更好的泛化性能。**
- **参数量差 10 倍**，一般没关系

----------------------------------------------
#### **总结**
- **FitNets 更深更窄，深有助于泛化，窄有助于减少计算**
- **一言以蔽之，KD 学习结果，FitNets 学习中间表示特征**

> **作者提出了另外四种架构，见原文，但是效果不如 FitNet**

- HT 和 KD 之间的**唯一区别**是参数空间的起点
- **HT 是比 KD 更强的正则化器，因为它在测试集上带来更好的泛化性能**

- **将窄而深的网络的内部层与教师网络的隐藏状态进行提示，比将它们与分类目标进行提示具有更好的泛化性能。**
- **参数量差 10 倍**，一般没关系
- **选择的 hint/guided layer 如果是卷积层，需要用卷积核来做 regressor，否则参数太多**
- 效果甚至会比 teacher model 更好
- 阶段训练，前面不是白折腾了？
    - 应该不是，其实是一种正则，约束参数空间的一种方式。

---------------------------------------

- **课程学习（Curriculum Learning）是一种模仿人类课程的训练策略**，它一开始在较简单的数据子集上训练模型，并逐渐拓展到更大更困难的数据子集，直到在整个数据集上训练。这种“由易到难”的训练策略在人类教育中很常见，例如，一个孩子要从最简单的加减乘除概念入手，逐步学习方程、求导等，才能学会微积分。
    - 在机器学习中，传统的训练策略是将所有数据集混合在一起，然后对模型进行训练。这种方法可能会导致模型在训练的早期阶段遇到困难，从而影响模型的性能。课程学习则可以通过逐步增加数据集的难度，帮助模型更好地学习。

    - 课程学习的**核心问题是如何定义数据的难度。一种常见的方法是使用人类专家的知识来定义数据的难度**。例如，在图像分类任务中，可以使用图像的清晰度、标签的一致性等因素来定义数据的难度。**另一种方法是使用数据驱动的方法来定义数据的难度**。例如，可以使用模型的预测准确率来定义数据的难度。

    - 课程学习**可以通过不同的方式来实现。一种常见的方法是使用预定义的课程**。在这种方法中，数据集会被预先划分为不同难度的数据集。模型会首先在较简单的数据集上进行训练，然后逐步增加数据集的难度。**另一种方法是使用自动生成的课程**。在这种方法中，模型会根据数据的难度来自动生成课程。

    - 课程学习已经在许多机器学习任务中取得了成功。例如，在图像分类、自然语言处理、机器翻译等任务中，课程学习都能够显著提高模型的性能。

- 以下是课程学习的一些**优势**：
    - 可以提高模型的泛化能力。
    - 可以加快模型的收敛速度。
    - 可以减少模型的过拟合风险。

- 课程学习也有一些**缺点**：
    - 需要额外的人工工作来定义数据的难度。
    - 在一些情况下，课程学习可能会降低模型的精度。

总体而言，课程学习是一种有效的训练策略，可以提高模型的性能。